# Common code for 'big paper' figures

In [ ]:
%matplotlib inline

import IPython.display as IPd
import delfi.distribution as dd
import delfi.utils.io as io
import delfi.utils.viz as viz
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.signal as ss
import socket
import svgutil

from delfi.utils.viz import plot_pdf
from math import factorial
from mpl_toolkits.axes_grid1 import make_axes_locatable
from svgutil.compose import Unit

In [ ]:
COL = {}
COL['GT']   = (35/255,86/255,167/255)
COL['SNPE'] = (0, 174/255,239/255)
COL['ESS']  = (244/255, 152/255, 25/255)
COL['SAMPLES']  = (244/255, 152/255, 25/255)
COL['IBEA']  = (102/255, 179/255, 46/255)
COL['EFREE'] = (105/255, 105/255, 105/255)
COL['I'] = (105/255, 105/255, 105/255)
COL['PRIOR'] = (244/255, 152/255, 25/255)

LABELS_HH =[r'$g_{Na}$', r'$g_{K}$', r'$g_{l}$', r'$g_{M}$', r'$t_{max}$', r'$-V_{T}$', r'$noise$', r'$-E_{l}$']
LABELS_HH_SPIKE_SUMSTATS = [r'$f\_rat$',r'$ap\_l$',r'$ap\_o$',r'$r\_pot$',r'$\sigma_{r\_pot}$',r'$ahd$',
                      r'$a\_ind$',r'$sp\_w$']
LABELS_HH_SUMSTATS = [r'$sp$',r'$r\_pot$',r'$\sigma_{r\_pot}$','$m_1$','$m_2$','$m_3$','$m_4$']

In [ ]:
# shorthands
sc = svgutil.compose
st = svgutil.transform

# conversion
def cm2cm(cm):
    return cm * 1

def cm2in(cm):
    return cm * 1 / Unit.per_inch['cm']

def cm2mm(cm):
    return cm * Unit.per_inch['mm'] / Unit.per_inch['cm']

def cm2pt(cm):
    return cm * Unit.per_inch['pt'] / Unit.per_inch['cm']

def cm2px(cm):
    return cm * Unit.per_inch['px'] / Unit.per_inch['cm']

In [ ]:
def svg(img):
    IPd.display(IPd.SVG(img))

In [ ]:
SCALE_PX = 6
SCALE_IN = 2.25
FIG_WIDTH_MM = 160

def mm2px(mm, scale=SCALE_PX):
    return scale*mm

def mm2inch(mm):
    return mm*0.0393701

mm2inches = mm2inch

# def create_fig(width_mm, height_mm):
#     return st.SVGFigure(mm2px(FIG_WIDTH_MM), mm2px(FIG_HEIGHT_MM))
def create_fig(width_mm, height_mm):
    return st.SVGFigure(mm2px(width_mm), mm2px(height_mm))

def add_label(fig, letter, x_pos_mm=0, y_pos_mm=0, font_size_px=18, weight='bold'):
    fig.append(st.TextElement(mm2px(x_pos_mm),  # location of letter in x direction
                              mm2px(y_pos_mm),  # location of letter in y direction
                              letter,  # letter
                              size=font_size_px,  # in px of font
                              weight=weight))
    return fig

def add_grid(fig, x_spacing_mm=10, y_spacing_mm=10, font_size_px=10, width_px=1):
    fig.append(sc.Grid(mm2px(x_spacing_mm),  # in mm, spacing of grid in x direction
                       mm2px(y_spacing_mm),  # in mm, spacing of grid in y direction
                       size=font_size_px,  # in px of font 
                       width=width_px,  # in px of grid lines
                       xmax=int(fig.width), 
                       ymax=int(fig.height),
                       multiply=1/SCALE_PX))
    return fig

def add_svg(fig, filename, x_pos_mm=0, y_pos_mm=0, scale=1, verbose=False):
    svg_file = st.fromfile(filename)
    width, height = svg_file.get_size()
    if verbose:
        print('size of svg of {} : {}'.format(filename, (width, height)))
    svg_root = svg_file.getroot()
    svg_root.moveto(mm2px(x_pos_mm), mm2px(y_pos_mm), scale=scale)
    fig.append([svg_root])
    return fig

def get_num(x):
    if type(x) == str:
        return float(''.join(ele for ele in x if ele.isdigit() or ele == '.'))
    else:
        return None

In [ ]:
HOSTNAME = socket.gethostname()

INKSCAPE = 'inkscape'
if HOSTNAME == 'nsa3004':  # jm workstation
    PATH_DROPBOX = '/home/jm/Mackelab/team/Write/Working_Manuscripts/InferenceNeuralDynamics_2018/'
elif HOSTNAME == 'nsa3010':  # pedro workstation
    PATH_DROPBOX = '/home/pedro/Mackelab/team/Write/Working_Manuscripts/InferenceNeuralDynamics_2018/'
elif HOSTNAME == 'Pep.local':  # pedro macbook
    PATH_DROPBOX = '/Users/pedro/Mackelab/team/Write/Working_Manuscripts/InferenceNeuralDynamics_2018/'
    INKSCAPE = '/Applications/Inkscape.app/Contents/Resources/script'
elif HOSTNAME == 'jml.local':  # jm macbook
    PATH_DROPBOX = '/Users/jm/Mackelab/team/Write/Working_Manuscripts/InferenceNeuralDynamics_2018/'
elif HOSTNAME == 'nsa2002.local' or HOSTNAME == 'nsa2002':  # no, not the nsa
    PATH_DROPBOX = '/Users/kaan/Dropbox/InferenceNeuralDynamics_2018/'
elif HOSTNAME == 'Bob': # Marcel
    PATH_DROPBOX = '/home/marcel/Dropbox (mackelab)/team/Write/Working_Manuscripts/InferenceNeuralDynamics_2018/'
else:
    raise ValueError('Unknown hostname {}, add in if-else block'.format(HOSTNAME))

PATH_DROPBOX_FIGS = PATH_DROPBOX + 'figs/'

MPL_RC = 'matplotlib.rc'

In [ ]:
dirs = {}
for model in ['glm', 'maprf', 'hodgkinhuxley', 'channelomics', 'l5pc']:
    dirs['dir_nets_{}'.format(model)] = '../results/'+model+'/nets/'
    dirs['dir_sampler_{}'.format(model)] = '../results/'+model+'/sampler/'
    dirs['dir_genetic_{}'.format(model)] = '../results/'+model+'/genetic/'

In [ ]:
dirs_dropbox = {}
for fig in ['fig1', 'fig2', 'fig3', 'fig4', 'fig5']:
    dirs_dropbox['dir_nets_{}'.format(fig)] = PATH_DROPBOX + 'results/'+fig+'/'